# Estimate channel bleed-through based on seq_qc data

There is an example in the starfish [BaristaSeq pipeline](https://spacetx-starfish.readthedocs.io/en/latest/gallery/pipelines/baristaseq_pipeline.html?highlight=registration#correct-for-bleed-through-from-illumina-sbs-reagents), with the method described in [Chen et al. 2017](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5829746/).

In order to estimate the bleed-through per channel, we use the first round of the AGTC test data after background filtering with a WhiteTophat filter.

In each of the experiments, only one of the four channels should have signal in the first round. Signal in all other channels is spurious. This is generally also true for later rounds. However, we expect some degree of imperfect incorporation and cleavage in the sequencing. Therefore, there might be true signal in other channels in later rounds.

Bleed-through factors for linear unmixing are estimated using Lasso regression as in [RoysamLab/whole_brain_analysis](https://github.com/RoysamLab/whole_brain_analysis) ([Maric et al. 2021](https://www.nature.com/articles/s41467-021-21735-x)).

Note: more sophisticated methods for bleed-through estimation exist but are not necessary in our case (e.g. [Theia by Ishaq et al. 2022](https://doi.org/10.1109/ISBI52829.2022.9761410))

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn import linear_model
from starfish import Experiment
from starfish.image import Filter
from starfish.types import Axes

## Background filtering

In [2]:
masking_radius = 6
filt = Filter.WhiteTophat(masking_radius, is_volume=False)
# now use it like this:
#filtered = filt.run(imgs, verbose=False, in_place=False)

## Channel bleed-through estimation


Based on the implementation in [RoysamLab/whole_brain_analysis:inter_channel_correction.py](https://github.com/RoysamLab/whole_brain_analysis/blob/7626c59d8696c5c80d4c8c5ac3f81c0e8170cb7f/RECONSTRUCTION/inter_channel_correction.py#L77-L96)

<details>

```python
def calculate_unmixing_params_unsupervised(images):
    # convert to float
    images = img_as_float(images)
    num_channels = images.shape[2]

    # make a list to keep result parameters
    results = np.zeros((num_channels, num_channels))

    # for all channels
    for i in range(num_channels):
        endmembers = [np.ndarray.flatten(images[:, :, idx]) for idx in range(num_channels)]
        source = endmembers.pop(i)

        clf = linear_model.Lasso(alpha=.0001, copy_X=True, positive=True)
        clf.fit(np.array(endmembers).T, source)
        alphas = np.insert(clf.coef_, i, 0)

        results[i, :] = alphas

    return results
```

</details>

In [3]:
# disclaimer: this value for the lasso alpha was chosen after experimentation with different values
lasso_alpha = 5e-5

In [4]:
results = []
for sample, sources in zip(["A_PB2", "C_PB1", "G_PA", "T_HA"], [[0,1,3,2,1,3], [3,1], [1,1], [2,0]]):
	exp = Experiment.from_json(os.path.join("data/spacetx/seq_qc/rep0/", sample, "experiment.json"))
	for fov in exp.fovs():
		imgs = fov.get_image("primary")
		# remove background with WhiteTophat filter
		filt.run(imgs, verbose=False, in_place=True)
		for round, source_channel in enumerate(sources):
			endmembers = [(1/imgs.xarray.max().item())*imgs.sel({Axes.ROUND: round}).xarray[0,idx,0].to_numpy().flatten() for idx in range(4)]
			source = endmembers.pop(source_channel)
			clf = linear_model.Lasso(alpha=lasso_alpha, copy_X=True, positive=True)
			#clf = linear_model.LinearRegression(copy_X=True, positive=True)
			clf.fit(np.array(endmembers).T, source)
			alphas = np.insert(clf.coef_, source_channel, 0)
			# print("AGTC"[source_channel], alphas)
			results.append({"sample": sample, "fov": fov.name, "round": round, "source": "AGTC"[source_channel], **dict(zip("AGTC", alphas))})

100%|██████████| 8/8 [00:00<00:00, 128.38it/s]


In [5]:
os.makedirs("analysis/bleed_through", exist_ok=True)
values = pd.DataFrame.from_records(results)
values.to_csv("analysis/bleed_through/lasso_results.csv", index=False)

### Estimated bleed-through factors

In [6]:
values[values["round"] == 0].groupby("source")[["A","G","T","C"]].median()

,A,G,T,C
source,,,,
A,0.0,0.0,0.655722,0.0
C,0.0,0.0,0.000000,0.0
G,0.0,0.0,0.000000,0.0
T,0.0,0.0,0.000000,0.0


## Bleed-through correction: linear unmixing

Starfish provides a method to correct for channel bleed-through via linear unmixing.
The coefficient matrix based on the results above (given the channel order "AGTC") is:

In [7]:
bleed = np.array(
    [[ 1.  ,  0.  , -0.656,  0.  ],
     [ 0.  ,  1.  ,  0.   ,  0.  ],
     [ 0.  ,  0.  ,  1.   ,  0.  ],
     [ 0.  ,  0.  ,  0.   ,  1.  ]]
)

In [8]:
lum = Filter.LinearUnmixing(bleed)
# now use it like this:
#bleed_corrected = lum.run(filtered, in_place=False)

### Verify that bleed-through from A to T is reduced

Load example fov from A_PB2 experiment:

In [9]:
def estimate_coefficients(data, round, source_channel):
    endmembers = [(1/data.xarray.max().item())*data.sel({Axes.ROUND: round}).xarray[0,idx,0].to_numpy().flatten() for idx in range(4)]
    source = endmembers.pop(source_channel)
    clf = linear_model.Lasso(alpha=lasso_alpha, copy_X=True, positive=True)
    #clf = linear_model.LinearRegression(copy_X=True, positive=True)
    clf.fit(np.array(endmembers).T, source)
    alphas = np.insert(clf.coef_, source_channel, 0)
    return alphas

In [10]:
exp = Experiment.from_json(os.path.join("data/spacetx/seq_qc/rep0/", "A_PB2", "experiment.json"))
fov = exp.fovs()[2]
imgs = fov.get_image("primary")
filtered = filt.run(imgs, verbose=False, in_place=False)
bleed_corrected = lum.run(filtered, in_place=False)

100%|██████████| 24/24 [00:00<00:00, 166.43it/s]


In [11]:
print("raw:", estimate_coefficients(imgs, 0, 0), sep="\t")
print("filtered:", estimate_coefficients(filtered, 0, 0), sep="\t")
print("corrected:", estimate_coefficients(bleed_corrected, 0, 0), sep="\t")

100%|██████████| 24/24 [00:00<00:00, 59.37it/s]


raw:	[0.       0.       7.089077 0.      ]
filtered:	[0.       0.       0.603057 0.      ]
corrected:	[0. 0. 0. 0.]
